* python 실행 경로 확인 
    - python 가상 환경을 사용 중이라면 가상 환경의 path 가 맞는지 확인한다. 


In [ ]:
%gui qt5
%matplotlib inline

import matplotlib.pyplot as plt

import sys
print(sys.executable)

## 1. 객체 생성

In [ ]:
from kw_condition import KiwoomOpenApiPlus
from PySide2.QtWidgets import QApplication

myApp = None
if isinstance(QApplication.instance(), type(None)):
    print('make instance')
    myApp = QApplication([])
else:
    print("use already")
    myApp = QApplication.instance()

kw_obj = KiwoomOpenApiPlus()

## 2. 서버접속
* 자동로그인이 설정되어 있는 경우 로그인이 자동으로 처리 되며, 그렇지 않은 경우 팝업 창에서 수동으로 아이디, 비밀번호 정보를 입력해야함 
- TODO
    * 자동 로그인 설정되지 않은 상태서 로그인 정보를 외부에서 입력한 경우 매크로로 자동처리 할수 있도록 해야함 pywinauto

In [ ]:
import time
kw_obj.tryConnect()

loop_count = 0
while kw_obj.isConnected() == False:
    loop_count = loop_count + 1
    time.sleep(1)
    QApplication.processEvents()
    if( loop_count > 30 ) :
        print('connect failed!')
        break



## 3. 서버 접속 상태 확인 

In [ ]:
kw_obj.isConnected()

## 4. TR 요청 예시 

아래 처럼 직접 TR 요청에 필요한 입력값을 설정해 요청하고, 이후 들어오는 이벤트 또한 직접 처리해주는 방식으로 사용할 수 있다.

In [9]:
rqname = '주식기본정보요청'
trcode = 'opt10001'

screen_no = '0001'  # 화면번호, 0000 을 제외한 4자리 숫자 임의로 지정, None 의 경우 내부적으로 화면번호 자동할당
inputs = {'종목코드': '005930'}

kw_obj.add_transaction(rqname, trcode, screen_no, inputs )
loop_count = 0
while True:
    loop_count = loop_count + 1
    time.sleep(1)
    QApplication.processEvents()
    if( loop_count > 7 ) :
        print('done!')
        break

* 16:38:57.650364 add_transaction  [{'rqname': '주식기본정보요청', 'trcode': 'opt10001', 'screen_no': '0001', 'inputs': {'종목코드': '005930'}}]
* 16:38:57.662490 tr_waiting_entered 
{'rqname': '주식기본정보요청', 'trcode': 'opt10001', 'screen_no': '0001', 'inputs': {'종목코드': '005930'}}
* 16:39:01.727709 tr_standby_entered 
done!
